In [127]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import datetime


In [128]:
data_usd_tr = pd.read_csv("csv files/USD_TRY.csv")
data_usd_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2870 entries, 0 to 2869
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tarih   2870 non-null   object
 1   Şimdi   2870 non-null   object
 2   Açılış  2870 non-null   object
 3   Yüksek  2870 non-null   object
 4   Düşük   2870 non-null   object
 5   Hac.    672 non-null    object
 6   Fark %  2870 non-null   object
dtypes: object(7)
memory usage: 157.1+ KB


In [129]:
df = data_usd_tr.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)

In [130]:
# We need to change from "," to "." because it will be a float.
df['Şimdi'] = df['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '.'))/10)
df

,Tarih,Şimdi
0,30.12.2022,1.86883
1,29.12.2022,1.86941
2,28.12.2022,1.87157
3,27.12.2022,1.86669
4,26.12.2022,1.86618
...,...,...
2865,06.01.2012,0.18776
2866,05.01.2012,0.18830
2867,04.01.2012,0.18785
2868,03.01.2012,0.18730


## LSTM

In [131]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[1.86883 1.86941 1.87157 1.86669 1.86618] 1.8663699999999999
[1.86941 1.87157 1.86669 1.86618 1.86637] 1.86424
[1.87157 1.86669 1.86618 1.86637 1.86424] 1.86749
[1.86669 1.86618 1.86637 1.86424 1.86749] 1.86479
[1.86618 1.86637 1.86424 1.86749 1.86479] 1.8642600000000003
[1.86637 1.86424 1.86749 1.86479 1.86426] 1.86343
[1.86424 1.86749 1.86479 1.86426 1.86343] 1.86368
[1.86749 1.86479 1.86426 1.86343 1.86368] 1.86232
[1.86479 1.86426 1.86343 1.86368 1.86232] 1.8575
[1.86426 1.86343 1.86368 1.86232 1.8575 ] 1.86325
[1.86343 1.86368 1.86232 1.8575  1.86325] 1.8650300000000002
[1.86368 1.86232 1.8575  1.86325 1.86503] 1.86324
[1.86232 1.8575  1.86325 1.86503 1.86324] 1.86355
[1.8575  1.86325 1.86503 1.86324 1.86355] 1.8623999999999998
[1.86325 1.86503 1.86324 1.86355 1.8624 ] 1.8622699999999999
[1.86503 1.86324 1.86355 1.8624  1.86227] 1.8630299999999997
[1.86324 1.86355 1.8624  1.86227 1.86303] 1.86313
[1.86355 1.8624  1.86227 1.86303 1.86313] 1.85895
[1.8624  1.86227 1.86303 1.86313 1.

In [153]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df['Şimdi'][0],df['Şimdi'][1],df['Şimdi'][2],df['Şimdi'][3],df['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Epoch 1/200
9/9 [==============================] - 2s 7ms/step - loss: 0.2744
Epoch 2/200
9/9 [==============================] - 0s 7ms/step - loss: 0.0410
Epoch 3/200
9/9 [==============================] - 0s 9ms/step - loss: 0.0192
Epoch 4/200
9/9 [==============================] - 0s 9ms/step - loss: 0.0106
Epoch 5/200
9/9 [==============================] - 0s 12ms/step - loss: 0.0058
Epoch 6/200
9/9 [==============================] - 0s 7ms/step - loss: 0.0042
Epoch 7/200
9/9 [==============================] - 0s 11ms/step - loss: 0.0040
Epoch 8/200
9/9 [==============================] - 0s 7ms/step - loss: 0.0035
Epoch 9/200
9/9 [==============================] - 0s 7ms/step - loss: 0.0032
Epoch 10/200
9/9 [==============================] - 0s 12ms/step - loss: 0.0030
Epoch 11/200
9/9 [==============================] - 0s 7ms/step - loss: 0.0028
Epoch 12/200
9/9 [==============================] - 0s 7ms/step - loss: 0.0027
Epoch 13/200
9/9 [==============================] - 0s 7ms

## Daily


In [133]:
print("Prediction: ",round(yhat[0][0]*10,2), "₺ USD/TL") # onluk olarak yukarı yuvarlandı.

Prediction:  18.85 ₺ USD/TL


## Weekly

In [134]:
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi']
                [2], df['Şimdi'][3], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi']
                [2], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],
                float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], float(yhat_4),
                float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)


yhat = float(yhat)*10
yhat_2 = float(yhat_2)*10
yhat_3 = float(yhat_3)*10
yhat_4 = float(yhat_4)*10
yhat_5 = float(yhat_5)*10


In [135]:
# onluk olarak yukarı yuvarlandı.
print(f"Pazartesi: {round(yhat,3)} Salı: {round(yhat_2,3)} Carsamba: {round(yhat_3,3)} Persembe: {round(yhat_4,3)} Cuma: {round(yhat_5,3)}  ₺ Pound/TRY")


Pazartesi: 18.852 Salı: 19.117 Carsamba: 18.87 Persembe: 18.898 Cuma: 18.887  ₺ Pound/TRY


## Monthly

In [136]:
import array
exchange = array.array('f', [])

for i in range(len(df)):
    if i % 5 == 0:  # it equals to weekly
        exchange.append(df['Şimdi'][i])


In [137]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.86883 1.86637 1.86343 1.86503 1.86303] 1.8585100173950195
[1.86637 1.86343 1.86503 1.86303 1.85851] 1.8588900566101074
[1.86343 1.86503 1.86303 1.85851 1.85889] 1.8570499420166016
[1.86503 1.86303 1.85851 1.85889 1.85705] 1.8565700054168701
[1.86303 1.85851 1.85889 1.85705 1.85657] 1.8581299781799316
[1.85851 1.85889 1.85705 1.85657 1.85813] 1.858839988708496
[1.85889 1.85705 1.85657 1.85813 1.85884] 1.8558499813079834
[1.85705 1.85657 1.85813 1.85884 1.85585] 1.8561999797821045
[1.85657 1.85813 1.85884 1.85585 1.8562 ] 1.850000023841858
[1.85813 1.85884 1.85585 1.8562  1.85   ] 1.8413100242614746
[1.85884 1.85585 1.8562  1.85    1.84131] 1.8254300355911255
[1.85585 1.8562  1.85    1.84131 1.82543] 1.8233400583267212
[1.8562  1.85    1.84131 1.82543 1.82334] 1.817770004272461
[1.85    1.84131 1.82543 1.82334 1.81777] 1.814329981803894
[1.84131 1.82543 1.82334 1.81777 1.81433] 1.808500051498413
[1.82543 1.82334 1.81777 1.81433 1.8085 ] 1.793239951133728
[1.82334 1.81777 1.81433 1.808

In [138]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array(
    [[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
18/18 [==============================] - 1s 7ms/step - loss: 0.1609
Epoch 2/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0122
Epoch 3/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0045
Epoch 4/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0025
Epoch 5/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0020
Epoch 6/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0016
Epoch 7/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0015
Epoch 8/300
18/18 [==============================] - 0s 9ms/step - loss: 0.0015
Epoch 9/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0013
Epoch 10/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0013
Epoch 11/300
18/18 [==============================] - 0s 8ms/step - loss: 0.0015
Epoch 12/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0014
Epoch 13/300
18/18 [=================

In [139]:
print(
    f"Next Month -> Prediction: {round(float(yhat[0][0]*10),3)}  ₺ USD/TRY in x.strftime('%B') ")


Next Month -> Prediction: 18.896  ₺ USD/TRY in January 


## Yearly

In [149]:
import array
exchange_2 = array.array('f', [])

for i in range(len(df)):
    if i % 10 == 0:  # it equals to weekly
        exchange_2.append(df['Şimdi'][i])


In [150]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange_2
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.86883 1.86343 1.86303 1.85889 1.85657] 1.858839988708496
[1.86343 1.86303 1.85889 1.85657 1.85884] 1.8561999797821045
[1.86303 1.85889 1.85657 1.85884 1.8562 ] 1.8413100242614746
[1.85889 1.85657 1.85884 1.8562  1.84131] 1.8233400583267212
[1.85657 1.85884 1.8562  1.84131 1.82334] 1.814329981803894
[1.85884 1.8562  1.84131 1.82334 1.81433] 1.793239951133728
[1.8562  1.84131 1.82334 1.81433 1.79324] 1.7911399602890015
[1.84131 1.82334 1.81433 1.79324 1.79114] 1.7395000457763672
[1.82334 1.81433 1.79324 1.79114 1.7395 ] 1.6761300563812256
[1.81433 1.79324 1.79114 1.7395  1.67613] 1.732550024986267
[1.79324 1.79114 1.7395  1.67613 1.73255] 1.641260027885437
[1.79114 1.7395  1.67613 1.73255 1.64126] 1.5895099639892578
[1.7395  1.67613 1.73255 1.64126 1.58951] 1.4946800470352173
[1.67613 1.73255 1.64126 1.58951 1.49468] 1.473389983177185
[1.73255 1.64126 1.58951 1.49468 1.47339] 1.474410057067871
[1.64126 1.58951 1.49468 1.47339 1.47441] 1.4828599691390991
[1.58951 1.49468 1.47339 1.4744

In [151]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array(
    [[exchange_2[0], exchange_2[1], exchange_2[2], exchange_2[3], exchange_2[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
9/9 [==============================] - 1s 9ms/step - loss: 0.2524
Epoch 2/300
9/9 [==============================] - 0s 8ms/step - loss: 0.0306
Epoch 3/300
9/9 [==============================] - 0s 9ms/step - loss: 0.0152
Epoch 4/300
9/9 [==============================] - 0s 10ms/step - loss: 0.0096
Epoch 5/300
9/9 [==============================] - 0s 9ms/step - loss: 0.0073
Epoch 6/300
9/9 [==============================] - 0s 10ms/step - loss: 0.0053
Epoch 7/300
9/9 [==============================] - 0s 8ms/step - loss: 0.0038
Epoch 8/300
9/9 [==============================] - 0s 8ms/step - loss: 0.0035
Epoch 9/300
9/9 [==============================] - 0s 8ms/step - loss: 0.0036
Epoch 10/300
9/9 [==============================] - 0s 15ms/step - loss: 0.0033
Epoch 11/300
9/9 [==============================] - 0s 8ms/step - loss: 0.0032
Epoch 12/300
9/9 [==============================] - 0s 8ms/step - loss: 0.0031
Epoch 13/300
9/9 [==============================] - 0s 9ms

In [152]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Year -> Prediction: {round(float(yhat[0][0]*10),3)}  ₺ USD/TL in x.strftime('%Y') ")


Next Year -> Prediction: 18.626  ₺ USD/TL in x.strftime('%Y') 
